In [1]:
# install libraries
!pip install gradio langchain-community langchain-openai langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 90.3 MB/s eta 0:00:

In [2]:
# imports libraries
import os
import requests
import json
from bs4 import BeautifulSoup
from openai import OpenAI
from google.colab import userdata
from tqdm import tqdm
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
import glob
import gradio as gr

In [3]:
# get the API key from Google Colab's Secrets
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

# feel free to change it to other models
MODEL = "gpt-4o"

In [4]:
# defines HTTP headers to simulate a legitimate browser user-agent string
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    a class to fetch and process content from a specified webpage URL.

    url (str): the URL of the webpage.
    body (bytes): raw HTML content of the webpage.
    title (str): the title of the webpage, if available.
    text (str): cleaned and visible text content of the webpage.
    links (list): a list of relevant links filtered by specific criteria.
    """
    def __init__(self, url):
        """
        Initializes the Website instance, fetches the webpage content, and processes it.

        url (str): The URL of the webpage to process.
        """
        self.url = url
        response = requests.get(url, headers=headers)

        # store the raw HTML content of the response
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')

        # extract the webpage title or provide a default message if no title is found
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose() # remove unnecessary tags
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]

        # filter out irrelevant links
        self.links = [link for link in links if link]
        self.links = [link for link in self.links if link.startswith("https://octopathtraveler.fandom.com/wiki/")]

    def get_links(self):
        """
        Returns a set of unique links found on the webpage.

        Returns:
            set: A set of unique links.
        """
        return set(self.links)

    def get_contents(self):
        """
        Returns the title and text content of the webpage.

        Returns:
            str: The title and text content of the webpage.
        """
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
all_links = Website("https://octopathtraveler.fandom.com/wiki/Octopath_Traveler_II").get_links()
all_links

{'https://octopathtraveler.fandom.com/wiki/Accessories',
 'https://octopathtraveler.fandom.com/wiki/Accessories_(Octopath_Traveler_II)',
 'https://octopathtraveler.fandom.com/wiki/Agnea_Bristarni',
 'https://octopathtraveler.fandom.com/wiki/Alfyn_Greengrass',
 'https://octopathtraveler.fandom.com/wiki/Apothecary',
 'https://octopathtraveler.fandom.com/wiki/Apothecary_(Octopath_Traveler_II)',
 'https://octopathtraveler.fandom.com/wiki/Blog:Recent_posts',
 'https://octopathtraveler.fandom.com/wiki/Body_Armor',
 'https://octopathtraveler.fandom.com/wiki/Body_Armor_(Octopath_Traveler_II)',
 'https://octopathtraveler.fandom.com/wiki/Brightlands',
 'https://octopathtraveler.fandom.com/wiki/Castti_Florenz',
 'https://octopathtraveler.fandom.com/wiki/Category:Bosses',
 'https://octopathtraveler.fandom.com/wiki/Category:Champions_of_the_Continent_Boss_Minions',
 'https://octopathtraveler.fandom.com/wiki/Category:Champions_of_the_Continent_Bosses',
 'https://octopathtraveler.fandom.com/wiki/Cate

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a knowledge base about this game, \
such as links to an Character page, Jobs page, Regions page,  etc.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "character", "url": "https://full.url/goes/here/character"},
        {"type": "location": "url": "https://another.full.url/location"}
    ]
}
"""


In [7]:
def get_links_user_prompt(website):
    """
    Generates a user-friendly prompt containing the list of links from a given website.

    Args:
        website: contains the URLs

    Returns:
        str: finished user prompt
    """
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a guide about the game, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
def get_links(url):
    """
    Fetches and processes links from a given URL using a chatbot model to determine relevant links.

    Args:
        url (str): The URL of the webpage to fetch and process.

    Returns:
        dict: A JSON object containing the filtered links as determined by the chatbot model.

    Raises:
        Exception: If the OpenAI API call fails or the response format is invalid.
    """
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [9]:
useful_links = get_links("https://octopathtraveler.fandom.com/wiki/Category:Octopath_Traveler_II")
useful_links

{'links': [{'type': 'location',
   'url': 'https://octopathtraveler.fandom.com/wiki/Orsterra'},
  {'type': 'location',
   'url': 'https://octopathtraveler.fandom.com/wiki/Frostlands'},
  {'type': 'location',
   'url': 'https://octopathtraveler.fandom.com/wiki/Flatlands'},
  {'type': 'location',
   'url': 'https://octopathtraveler.fandom.com/wiki/Coastlands'},
  {'type': 'location',
   'url': 'https://octopathtraveler.fandom.com/wiki/Highlands'},
  {'type': 'location',
   'url': 'https://octopathtraveler.fandom.com/wiki/Sunlands'},
  {'type': 'location',
   'url': 'https://octopathtraveler.fandom.com/wiki/Riverlands'},
  {'type': 'location',
   'url': 'https://octopathtraveler.fandom.com/wiki/Cliftlands'},
  {'type': 'location',
   'url': 'https://octopathtraveler.fandom.com/wiki/Woodlands'},
  {'type': 'character',
   'url': 'https://octopathtraveler.fandom.com/wiki/Ophilia_Clement'},
  {'type': 'character',
   'url': 'https://octopathtraveler.fandom.com/wiki/Cyrus_Albright'},
  {'type

In [10]:
def message_for(file_name, content):
  system_prompt = (
    f"You are a specialized assistant tasked with gathering relevant and accurate information "
    f"about {file_name} related to Octopath Traveler (video game). When provided with contents, "
    f"your role is to extract and present all relevant details about {file_name}, focusing solely "
    f"on gathering information without summarizing, interpreting, or analyzing. Respond in markdown.")

  return [
    {"role": "system",
     "content": system_prompt},
    {"role": "user",
     "content": content}
  ]


In [11]:
def create_knowledge_base(useful_links, knowledge_base):
  if not os.path.exists(knowledge_base):
    os.mkdir(knowledge_base)
  for i in tqdm(range(len(useful_links["links"]))):
    folder_path = f"{knowledge_base}/{useful_links['links'][i]['type']}"
    file_name = useful_links['links'][i]['url'].split("/")[-1]
    file_path = os.path.join(folder_path, file_name) + ".md"
    if not os.path.exists(folder_path):
      os.mkdir(folder_path)
      print(f"{folder_path} created.")

    # uncommon the line below if more URLs are scrapped
    # if not os.path.exists(file_path):
    print(f"Creating knowledge for {file_name}")
    content = Website(useful_links['links'][i]['url']).get_contents()
    messages = message_for(file_name, content)
    response = openai.chat.completions.create(
      model=MODEL,
      messages=messages
    )

    # write knowledge into files
    with open(file_path, "w") as f:
      f.write(response.choices[0].message.content)
    print(f"\nDone!")

In [12]:
create_knowledge_base(useful_links, "octopath-knowledge")

  0%|          | 0/47 [00:00<?, ?it/s]

octopath-knowledge/location created.
Creating knowledge for Orsterra


  2%|▏         | 1/47 [00:14<10:57, 14.28s/it]


Done!
Creating knowledge for Frostlands


  4%|▍         | 2/47 [00:26<09:36, 12.80s/it]

...


Done!
Creating knowledge for Category:Octopath_Traveler_II_Bosses


 98%|█████████▊| 46/47 [13:01<00:12, 12.38s/it]


Done!
Creating knowledge for Side_Stories_(Octopath_Traveler_II)


100%|██████████| 47/47 [13:23<00:00, 17.10s/it]


Done!


In [22]:
db_name = "octopath_db"

# take everything in all the sub-folders of the knowledge base
folders = glob.glob("octopath-knowledge/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

# try this code if the line below doesn't work: text_loader_kwargs={'autodetect_encoding': True}
text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
chunks = text_splitter.split_documents(documents)

document_types = list(set([doc.metadata['doc_type'] for doc in documents]))
print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {document_types}")

Total number of chunks: 109
Document types found: ['page', 'character', 'location']


In [23]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
# An alternative approach
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete DB if already exists
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 109 documents


In [24]:
import random
from matplotlib import colors

def generate_random_colors(num_colors):
    color_names = list(colors.CSS4_COLORS.keys())
    random_colors = random.sample(color_names, num_colors)
    return random_colors

num_colors = len(set(doc.metadata['doc_type'] for doc in documents))
random_colors = generate_random_colors(num_colors)
print(random_colors)

['mediumblue', 'slateblue', 'lightblue']


In [25]:
# prework for visualizing vectorstore

collection = vectorstore._collection
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [random_colors[document_types.index(t)] for t in doc_types]

In [31]:
# visualize vectorstore in 2D

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [32]:
# 3D

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [28]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.5, model_name=MODEL, openai_api_key=userdata.get("OPENAI_API_KEY"))

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# putting it together: set up the conversation chain with the LLM model, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

<ipython-input-28-fd62d028c018>:5: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [29]:
# Handles a conversational interaction by processing a user's question within a conversation context.
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [30]:
# launch the Gradio interface, ask the Chatbot questions about Octopath Traveler 2
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://306b90f4a01cceedbf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
